In [1]:
import pandas as pd
from pathlib import Path
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
# # create engine to the database

# engine = create_engine("sqlite:///Philly_Bridges.sqlite")
# query = "SELECT * FROM bridges"

# #Extract the data from the database and covert it to a pandas data frame

# bridge_df = pd.read_sql_query(query, engine)

# bridge_df

In [3]:
bridge_df = pd.read_csv("Cleaned_Bridge_df.csv")
# bridge_df

In [4]:
myMap = folium.Map(location = (39.952583, -75.165222), tiles = "OpenStreetMap", zoom_start=11.5)

#Create a function to assign differnet colors dependant on the amount of ADT on each bridge

def get_circle_color(value):
    if value <= 24999:
        return 'green'
    elif value >= 25000 and value < 49999:
        return 'orange'
    else:
        return 'red'

#Create separate groups for each cirlce color

traffic_volume = folium.FeatureGroup(name = "Traiffic Volume", show=False )
# med_traffic = folium.FeatureGroup(name = "Medium Traffic")
# high_traffic = folium.FeatureGroup(name = "High Traffic") 

# Iterate over your DataFrame
for index, row in bridge_df.iterrows():
    # Get the value from your column
    value = row["Average Daily_Traffic"]

    # Determine the color based on the value ranges
    color = get_circle_color(value)

    # Add a circle to the map with the determined color
    marker = folium.CircleMarker(location=[row["Latitude"], row["Longitude"]], color=color, radius=10).add_to(traffic_volume)
    
#     if value < 25000:
#         marker.add_to(low_traffic)
#     elif value >= 25000 and value < 50000:
#         marker.add_to(med_traffic)
#     else:
#         marker.add_to(high_traffic)   


# Add the three traffic level groups to the map    
    
# low_traffic.add_to(myMap)
# med_traffic.add_to(myMap)
# high_traffic.add_to(myMap)


# Create a function to assign different colors for the three different condition grades    

condition = bridge_df["Bridge_Condition"]
def get_marker_color(condition):
    if condition == "Good":
        return "green"
    elif condition == "Fair":
        return "orange"
    elif condition == "Poor":
        return "darkred"
    else:
        return "gray"

#Create separate groups for each marker type
good_group = folium.FeatureGroup(name="Good Condition")
fair_group = folium.FeatureGroup(name="Fair Condition")
poor_group = folium.FeatureGroup(name="Poor Condition")

#Markers
for index, row in bridge_df.iterrows():
    ID = row["Bridge_ID"]
    lat = row["Latitude"]
    long = row["Longitude"]
    condition = row["Bridge_Condition"]
    address = row["Address"]
    repair_cost = row["Repair_Cost_Est"]
    replace_cost = row["Replacement_Cost_Est"]
    #Marker Coloring
    marker_color = get_marker_color(condition)
    text = f"ID: {ID} <br>Address: {address} <br>Latitude: {lat} <br>Longitude: {long} <br> Condition: {condition} <br> Estimaed Cost to Repair: {repair_cost} <br> Estimaed Cost to Replace: {replace_cost}"
    popup = folium.Popup(text, max_width = 300)
    marker = folium.Marker([lat, long], popup=popup, icon=folium.Icon(color=marker_color))
    
    if condition == "Good":
        marker.add_to(good_group)
    elif condition == "Fair":
        marker.add_to(fair_group)
    elif condition == "Poor":
        marker.add_to(poor_group)
    else:
        marker.add_to(myMap)

good_group.add_to(myMap)
fair_group.add_to(myMap)
poor_group.add_to(myMap)
traffic_volume.add_to(myMap)


folium.LayerControl().add_to(myMap)

myMap